Zbiór danych `IMDB` zawiera pozytywne i negatywne recenzje filmów. Zwektoryzuj teksty recenzji, a następnie użyj metody [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html), aby porównać:
    
- dokładność (accuracy) i f1-score,
- czas trenowania (fit time),
- czas testowania (score time)

wybranych modeli klasyfikacyjnych, w tym `MultinomialNB`, `KNeighborsClassifier` oraz `LogisticRegression`.

In [14]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate

from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [15]:
df = pd.read_csv('../data/IMDB_Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
y = df['sentiment'].map({'positive': 1, 'negative': 0})
print(y.value_counts())

sentiment
1    25000
0    25000
Name: count, dtype: int64


In [17]:
cv = CountVectorizer(stop_words="english")
X = cv.fit_transform(df['review'])

print("df shape:", df.shape)
print("X shape:", X.shape)

print("Vocabulary size:", len(cv.vocabulary_))

df shape: (50000, 2)
X shape: (50000, 101583)
Vocabulary size: 101583


In [18]:
print(X[:5].toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [19]:
# Labels for confusion matrix
# negative -> index 0, positive -> index 1
labels_for_cm = [0, 1]
target_names = ['ham', 'spam']

In [20]:
# Metrics
metrics = ['accuracy', 'f1_macro']

### Multinomial Native Bayes (MultinomialNB)

In [21]:
mnb = MultinomialNB()

cv_results = cross_validate(
    mnb, 
    X, 
    y, 
    cv=5, 
    scoring=metrics,
    n_jobs=-1
)

for key, value in cv_results.items():
    print(f"  '{key}': {value}")

  'fit_time': [0.02409792 0.02387977 0.02079988 0.02253795 0.02149606]
  'score_time': [0.00528598 0.00532007 0.01057196 0.00533605 0.00491381]
  'test_accuracy': [0.858  0.8616 0.8559 0.8534 0.8583]
  'test_f1_macro': [0.85794088 0.86149449 0.85581203 0.85336056 0.85821489]


In [22]:
print(f"\nCzasy trenowania (fit_time) dla każdego foldu: {cv_results['fit_time']}")
print(f"Średni czas trenowania: {np.mean(cv_results['fit_time']):.4f} s +/- {np.std(cv_results['fit_time']):.4f} s")

print(f"\nCzasy testowania/oceny (score_time) dla każdego foldu: {cv_results['score_time']}")
print(f"Średni czas testowania: {np.mean(cv_results['score_time']):.4f} s +/- {np.std(cv_results['score_time']):.4f} s")

accuracy_mnb_key = f"test_{metrics[0]}"
print(f"\nDokładność (accuracy) dla każdego foldu: {cv_results[accuracy_mnb_key]}")
print(f"Średnia dokładność: {np.mean(cv_results[accuracy_mnb_key]):.4f} +/- {np.std(cv_results[accuracy_mnb_key]):.4f}")

f1_mnb_key = f"test_{metrics[1]}"
print(f"\nF1-score ({metrics[1]}) dla każdego foldu: {cv_results[f1_mnb_key]}")
print(f"Średni F1-score ({metrics[1]}): {np.mean(cv_results[f1_mnb_key]):.4f} +/- {np.std(cv_results[f1_mnb_key]):.4f}")


Czasy trenowania (fit_time) dla każdego foldu: [0.02409792 0.02387977 0.02079988 0.02253795 0.02149606]
Średni czas trenowania: 0.0226 s +/- 0.0013 s

Czasy testowania/oceny (score_time) dla każdego foldu: [0.00528598 0.00532007 0.01057196 0.00533605 0.00491381]
Średni czas testowania: 0.0063 s +/- 0.0021 s

Dokładność (accuracy) dla każdego foldu: [0.858  0.8616 0.8559 0.8534 0.8583]
Średnia dokładność: 0.8574 +/- 0.0027

F1-score (f1_macro) dla każdego foldu: [0.85794088 0.86149449 0.85581203 0.85336056 0.85821489]
Średni F1-score (f1_macro): 0.8574 +/- 0.0027


### K-Nearest Neighbors (KNN)

In [23]:
knn = KNeighborsClassifier(n_neighbors=5)

cv_results = cross_validate(
    knn, 
    X, 
    y, 
    cv=5, 
    scoring=metrics,
    n_jobs=-1
)

for key, value in cv_results.items():
    print(f"  '{key}': {value}")

  'fit_time': [0.01243019 0.01354504 0.01338482 0.01523566 0.02100086]
  'score_time': [16.88444185 16.87247205 16.93097401 16.93297625 16.89775205]
  'test_accuracy': [0.5898 0.5981 0.6003 0.6016 0.5907]
  'test_f1_macro': [0.57943346 0.59215836 0.59748007 0.59562062 0.58167266]


In [24]:
print(f"\nCzasy trenowania (fit_time) dla każdego foldu: {cv_results['fit_time']}")
print(f"Średni czas trenowania: {np.mean(cv_results['fit_time']):.4f} s +/- {np.std(cv_results['fit_time']):.4f} s")

print(f"\nCzasy testowania/oceny (score_time) dla każdego foldu: {cv_results['score_time']}")
print(f"Średni czas testowania: {np.mean(cv_results['score_time']):.4f} s +/- {np.std(cv_results['score_time']):.4f} s")

accuracy_knn_key = f"test_{metrics[0]}"
print(f"\nDokładność (accuracy) dla każdego foldu: {cv_results[accuracy_knn_key]}")
print(f"Średnia dokładność: {np.mean(cv_results[accuracy_knn_key]):.4f} +/- {np.std(cv_results[accuracy_knn_key]):.4f}")

f1_knn_key = f"test_{metrics[1]}"
print(f"\nF1-score ({metrics[1]}) dla każdego foldu: {cv_results[f1_knn_key]}")
print(f"Średni F1-score ({metrics[1]}): {np.mean(cv_results[f1_knn_key]):.4f} +/- {np.std(cv_results[f1_knn_key]):.4f}")


Czasy trenowania (fit_time) dla każdego foldu: [0.01243019 0.01354504 0.01338482 0.01523566 0.02100086]
Średni czas trenowania: 0.0151 s +/- 0.0031 s

Czasy testowania/oceny (score_time) dla każdego foldu: [16.88444185 16.87247205 16.93097401 16.93297625 16.89775205]
Średni czas testowania: 16.9037 s +/- 0.0244 s

Dokładność (accuracy) dla każdego foldu: [0.5898 0.5981 0.6003 0.6016 0.5907]
Średnia dokładność: 0.5961 +/- 0.0049

F1-score (f1_macro) dla każdego foldu: [0.57943346 0.59215836 0.59748007 0.59562062 0.58167266]
Średni F1-score (f1_macro): 0.5893 +/- 0.0074


### Logistic Regression

In [25]:
lr = LogisticRegression(max_iter=1000, solver='liblinear')

cv_results = cross_validate(
    lr, 
    X, 
    y, 
    cv=5, 
    scoring=metrics,
    n_jobs=-1
)

for key, value in cv_results.items():
    print(f"  '{key}': {value}")

  'fit_time': [2.84814906 2.62763095 2.70806909 2.39779568 2.86651611]
  'score_time': [0.00398517 0.00285792 0.00317192 0.0032661  0.00335693]
  'test_accuracy': [0.8887 0.8823 0.8854 0.8809 0.8868]
  'test_f1_macro': [0.888695   0.88229644 0.88539641 0.88088785 0.88679984]


In [26]:
print(f"\nCzasy trenowania (fit_time) dla każdego foldu: {cv_results['fit_time']}")
print(f"Średni czas trenowania: {np.mean(cv_results['fit_time']):.4f} s +/- {np.std(cv_results['fit_time']):.4f} s")

print(f"\nCzasy testowania/oceny (score_time) dla każdego foldu: {cv_results['score_time']}")
print(f"Średni czas testowania: {np.mean(cv_results['score_time']):.4f} s +/- {np.std(cv_results['score_time']):.4f} s")

accuracy_lr_key = f"test_{metrics[0]}"
print(f"\nDokładność (accuracy) dla każdego foldu: {cv_results[accuracy_lr_key]}")
print(f"Średnia dokładność: {np.mean(cv_results[accuracy_lr_key]):.4f} +/- {np.std(cv_results[accuracy_lr_key]):.4f}")

f1_lr_key = f"test_{metrics[1]}"
print(f"\nF1-score ({metrics[1]}) dla każdego foldu: {cv_results[f1_lr_key]}")
print(f"Średni F1-score ({metrics[1]}): {np.mean(cv_results[f1_lr_key]):.4f} +/- {np.std(cv_results[f1_lr_key]):.4f}")


Czasy trenowania (fit_time) dla każdego foldu: [2.84814906 2.62763095 2.70806909 2.39779568 2.86651611]
Średni czas trenowania: 2.6896 s +/- 0.1707 s

Czasy testowania/oceny (score_time) dla każdego foldu: [0.00398517 0.00285792 0.00317192 0.0032661  0.00335693]
Średni czas testowania: 0.0033 s +/- 0.0004 s

Dokładność (accuracy) dla każdego foldu: [0.8887 0.8823 0.8854 0.8809 0.8868]
Średnia dokładność: 0.8848 +/- 0.0029

F1-score (f1_macro) dla każdego foldu: [0.888695   0.88229644 0.88539641 0.88088785 0.88679984]
Średni F1-score (f1_macro): 0.8848 +/- 0.0029
